In [3]:
import torch
import torch.nn as nn
import torch.utils.data as dt
from PIL import Image
import torchvision.transforms as transforms
import os
import numpy as np

# <font size=10 color=blue> Еще задачи "Computer Vision" </font>
![Segmentation](./img/Segment01.png)

# <font size=10 color=blue> Семантическа сегментация (Semantic Segmentation) </font>
![Segmentation](./img/Segment02.png)

<font color=blue size=5>Делаем предположение только о пикселях, каким классам эти пиксели принадлежат</font>

## Делаем сверточную сеть и решаем задачу классификации

![Segmentation](./img/Segment03.png)

<font color=red size=5>Основная проблема - это тяжелые сверточные слои. Каждая карта признаков слоя повторяет размер картинки</font>

## Делаем сверточную сеть cо сверточными слоями:

- <font color=red size=4>Downsampling</font>
- <font color=blue size=4>Upsampling</font>

![Segmentation](./img/Segment04.png)


# Upsampling - unpooling
![Segmentation](./img/Segment05.png)

# "Max unpooling"
![Segmentation](./img/Segment06.png)


# Unpooling с параметрами

## Свертка 3x3 stride 2 уменьшает входное изображение в два раза
![Segmentation](./img/Segment07.png)
### _pooling_ с параметрами
---

## Транспонированная конволюция 3x3 stride 2 
![Segmentation](./img/Segment08.png)

### _Stride_ задает соотношение в преобразовании

# Пример транспонированной 1D конволюции
![Segmentation](./img/Segment09.png)

# Конволюция как операция перемножения матриц (1D)
![Segmentation](./img/Segment10.png)

# Практика
Реализовать сверточную сеть для семантической сегментации
- В файле model.py   - имплементировать заглушку модели
- В в файле train.py - нужно поставить правильный loss

### Зависимости
 - tensorflow  - поддержка tensorboard
 - tensorboardx - тензор боард для pytorch
 - tqdm         - пакет для отрисовки прогресс баров

### Запуск пакета
_ По умолчанию все данные лежат в папке ./data/. Если вы положили их в другую папку, то поправте в скрипте train.py пути _
<br/>
python train.py

# <font size=10 color=blue> SegNet  </font>
### _[V. Badrinarayanan, 2016]_
![Segmentation](./img/Segment11.png)

<font color=blue size=5>При сегментации использовали перенос макс пулинга</font>

# <font size=10 color=blue> U-Net  </font>
### _[O. Ronneberger, 2017]_
![Segmentation](./img/Segment12.png)

<font color=blue size=5>Используем "shortcut" для соедиение симметричных слоев из конволюций и деконволюций</font>

# Практика
Реализовать сверточную сеть для семантической сегментации по типу U-Net. Принцип как в предыдущем разделе практики - реализуем модель в model.py

# <font size=10 color=blue> Pyramid Scene Parsing Network  </font>
### _[H. Zhao, 2016]_

![Segmentation](./img/Segment13.png)
<font color=blue size=5>Обычная "feedforward" конволюционная сеть слепа к мелким деталям. Рецептивное поле на верхних слоях намного больше, чем само входное изображение. Это приводит к потере  детализации изображения</font>

![Segmentation](./img/Segment14.png)
===
<font color=blue size=5>Нужно рассмотреть картинку в различных разрешениях, организовав глобальный пулинг "Pyramid Pooling Module". В результате сеть получает возможность видеть больше деталей</font>

- CNN - предобученный ResNet-101
- Выходные панели пулинга апсамплятся и конкатенируются
- Выход Spatial Softmax

# <font size=10 color=blue> Atrous Spatial Pyramid Pooling </font>
### _[Chen, 2017]_

![Segmentation](./img/Segment15.png)

## Архитектуры сетей с расширением захвата разномаштабных элементов изображнения


## Atrous convolution
![Segmentation](./img/Segment16.png)

$$ y[i] = \sum_{k} x[i+r*k] w[i] $$

* i - локация
* y - выход 
* w - фильтр 
* x - вход
* r - шаг дырки


- Задача семантической сегментации не потерять детали
- Пулинг заставляет сеть выучивать абстрактные представления фичей, теряя детали
- Комбинации Atrous конволюций позволяют видеть изображение с разным разшением без увеличения количества параметров

![Segmentation](./img/Segment17.png)
## Atrous Spatial Pyramid Pooling
В качестве базовой сети использовался ResNet.


In [23]:
input = torch.autograd.Variable( torch.rand((1,3,224,224)))

atrous_conv = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1, dilation=6)
conv        = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1, dilation=1)
o = conv(input)
print ("General convolution - ", o.size(), conv.weight.size())
o = atrous_conv(input)
print ("Atrous convolution - ", o.size(), atrous_conv.weight.size())


General convolution -  torch.Size([1, 32, 224, 224]) torch.Size([32, 3, 3, 3])
Atrous convolution -  torch.Size([1, 32, 214, 214]) torch.Size([32, 3, 3, 3])


## WAVENET: A GENERATIVE MODEL FOR RAW AUDIO
[WaveNET](https://arxiv.org/pdf/1609.03499.pdf)

Генерирует звук в авторегрессионном режиме, так как чувствительна к размеру рецептивного поля, оно определяет качество синтезированного звука, авторы используют _Artrous 1d конволюции_
![Segmentation](./img/WaveNET.png)